# Seq2Seq Machine Translation with Attention

## 1. Preprocessing

Dataset link: https://www.statmt.org/wmt13/training-parallel-europarl-v7.tgz

In [1]:
# Check the size of both datasets
with open("./data/unprocessed/europarl-v7.de-en.de") as file:
    ger = [line.rstrip() for line in file]
with open("./data/unprocessed/europarl-v7.de-en.en") as file:
    eng = [line.rstrip() for line in file]

print(len(eng))
print(len(ger))

1920209
1920209


In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
# Load spacy data
# !python -m spacy download de_core_news_sm
# !python -m spacy download en_core_web_sm

In [6]:
from seq2seq_attention.preprocess import get_parallel_csv

# Take ">" as seperator since it is not included in the text - unique to seperate eng-ger pairs. 
# Remove any ">" from text pairs.
get_parallel_csv(path_1="./data/unprocessed/europarl-v7.de-en.de", path_2="./data/unprocessed/europarl-v7.de-en.en", new_file_path="./data/processed/en_ger_full.csv", delimiter=">")

File successfully created.
11289 lines were removed.


In [40]:
# Remove sentences with lower number of words
from seq2seq_attention.preprocess import remove_sentences
remove_sentences(data_dir="./data/processed/en_ger_full.csv", min_length=4, max_length=30, delimiter=">", new_file_path="./data/processed/en_ger_full_removed_sent_len.csv")

File successfully created.
824619 sentence-pairs were removed.


In [41]:
from seq2seq_attention.preprocess import train_test_split
train_test_split(file_path="./data/processed/en_ger_full_removed_sent_len.csv", sep=">", random_seed=118, dir="./data/processed")

All files successfully created.


In [43]:
# Check files
import pandas as pd
train = pd.read_csv("./data/processed/train.csv", header=None, sep=">", names=["ger", "eng"])
val = pd.read_csv("./data/processed/val.csv", header=None, sep=">", names=["ger", "eng"])
test = pd.read_csv("./data/processed/test.csv", header=None,sep=">", names=["ger", "eng"])

In [44]:
train_len = len(train)
val_len = len(val)
test_len = len(test)
print(f"Train: {train_len}, Val: {val_len}, Test: {test_len}")
print(f"Total: {train_len+val_len+test_len}")
print(f"Total + Removed: {train_len+val_len+test_len+11289+824619}")

Train: 867441, Val: 108430, Test: 108430
Total: 1084301
Total + Removed: 1920209


In [11]:
train.head()

,ger,eng
0,Indessen ist weder bei Bulgarien noch bei Rumä...,Neither Bulgaria nor Romania’s accession proc...
1,Seit Amsterdam stehen drei Begriffe im Mittelp...,"Since Amsterdam, three terms have taken centr..."
2,Um die Kyoto-Ziele zu erfüllen und die Umwelt ...,In order to meet the Kyoto objectives and to ...
3,Das irische Volk hat die spezifische Version d...,The Irish people rejected the particular vers...
4,Es gab eindeutige Hinweise auf organisierte An...,There were clear signs of organised attacks o...


In [12]:
val.head()

,ger,eng
0,Wir können die Änderungsanträge 49 und 50 nich...,"We cannot accept Amendments Nos 49 and 50, wh..."
1,"Daher schlage ich vor, dass wir uns unter hohe...","That is why, while keenly appreciating the ri..."
2,"Deshalb sollten wir versuchen, diesem anderen ...","It is to this other Iran, then, that we shoul..."
3,"Es besteht kein Mangel an Kreativität, was den...",There is no lack of creativity when it comes ...
4,"Ich bitte dringend darum, den Themen mehr Aufm...",I would urge that more attention be paid to i...


In [13]:
test.head()

,ger,eng
0,Sie haben Regulierungsinstrumente für die Prod...,They have destroyed regulatory instruments fo...
1,Keine andere Meinung zählt.,Nobody else has a say.
2,"Deshalb müssen wir dagegen protestieren, dass ...",That is why we must protest against the fact ...
3,Die Aussprache ist hiermit geschlossen.,That concludes the debate.
4,"Wie meine Vorredner und Vorrednerinnen, Herr K...","Like the previous speakers, Commissioner, I t..."


## 2. Initialize Dataloaders

In [14]:
from seq2seq_attention.build_dataloaders import build_fields, build_bucket_iterator, get_datasets, build_vocab
BATCH_SIZE = 100
DEVICE = "cpu"

src_field, trg_field = build_fields()
train_set, val_set, test_set = get_datasets(train_path="./data/processed/train.csv", 
                                            val_path="./data/processed/val.csv", 
                                            test_path="./data/processed/test.csv", 
                                            src_field=src_field, 
                                            trg_field=trg_field)
build_vocab(src_field=src_field, trg_field=trg_field, train_set=train_set, min_freq=2, max_vocab_size=32000)

In [33]:
# Check vocabulary 
len(src_field.vocab)
src_field.vocab.stoi["<pad>"]

1

In [35]:
src_field.vocab.

In [15]:
train_loader = build_bucket_iterator(dataset=train_set, batch_size=BATCH_SIZE, device=DEVICE)
val_loader = build_bucket_iterator(dataset=val_set, batch_size=BATCH_SIZE, device=DEVICE)
test_loader = build_bucket_iterator(dataset=test_set, batch_size=BATCH_SIZE, device=DEVICE)

In [16]:
# Retrieve sample batch
iterator = iter(train_loader)

In [17]:
example = next(iterator)
src_batch = example.src
trg_batch = example.trg
print(src_batch[0].shape, src_batch[1].shape)
print(trg_batch[0].shape, trg_batch[1].shape)

torch.Size([100, 6]) torch.Size([100])
torch.Size([10]) torch.Size([10])


In [18]:
print(src_batch[1])

tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6])


In [19]:
print(trg_batch[1])

tensor([   2,   17,   13, 3232,    6,    3,    1,    1,    1,    1])


In [21]:
# itos is list of token strings with their idx 
for j in range(5):
    src = ""
    for i in src_batch[0][j]:
       src = " ".join([src,  src_field.vocab.itos[i]])
    print(src)
    trg = ""
    for i in trg_batch[j]:
        trg = " ".join([trg, trg_field.vocab.itos[i]])
    print(trg)
    print()
# The second element in the tuple is the real length that we pass to the packed_seq!

 <sos> gibt es einwände ? <eos>
 <sos> are there any comments ? <eos> <pad> <pad> <pad>

 <sos> das ist absurd . <eos>
 <sos> this is absurd . <eos> <pad> <pad> <pad> <pad>

 <sos> - vor der abstimmung <eos>
 <sos> - before the vote : <eos> <pad> <pad> <pad>

 <sos> <unk> ( fortsetzung ) <eos>
 <sos> alpine transit ( continuation ) <eos> <pad> <pad> <pad>

 <sos> das ist bedauernswert . <eos>
 <sos> that is regrettable . <eos> <pad> <pad> <pad> <pad>



In [31]:
# Create small trainig set to test computaitons 
with open("./data/processed/train.csv", "r") as file:
        with open("./data/processed/train_mini.csv", "w") as new_file:
            i = 0
            for line in file:
                if i < 1000:
                    new_file.write(line)
                    
                i+=1
print("File successfully created.")

File successfully created.
